# Alexnet Model on Custom Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten,Dense, Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D , GlobalMaxPooling2D

from keras.preprocessing import image

from keras.utils import layer_utils

from keras.utils.data_utils import get_file

from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

# from keras.applications.imagenet_utils import _obtain_input_shape # this will work for older versions of keras. 2.2.0 or before

# from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import get_source_inputs

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os



In [ ]:

img_dir = '/content/drive/MyDrive/BTdsYesNo/'

dataset_path = os.listdir(img_dir)

dataset_category = os.listdir(img_dir)
print (dataset_category)  #what kinds of data are in this dataset

print("Types of data found: ", len(dataset_path))

['yes', 'no']
Types of data found:  2


In [ ]:
dataset = []

for item in dataset_category:
    # Get all the file names
    all_data = os.listdir(img_dir  +item)

    print(all_data)

 # Add them to the list
    for images in all_data:
        dataset.append((item, str(img_dir+ '/' +item) + '/' + images))
        # print(dataset)

['y1333.jpg', 'y1096.jpg', 'y1082.jpg', 'y204.jpg', 'y1408.jpg', 'y1158.jpg', 'y132.jpg', 'y108.jpg', 'y1344.jpg', 'y1422.jpg', 'y1222.jpg', 'y1.jpg', 'y1237.jpg', 'y1165.jpg', 'y1409.jpg', 'y1475.jpg', 'y1401.jpg', 'y1118.jpg', 'y1063.jpg', 'y1268.jpg', 'y1284.jpg', 'y14.jpg', 'y1411.jpg', 'y1270.jpg', 'y1164.jpg', 'y1130.jpg', 'y1208.jpg', 'y1135.jpg', 'y1252.jpg', 'y1286.jpg', 'y1012.jpg', 'y1271.jpg', 'y1352.jpg', 'y1115.jpg', 'y1143.jpg', 'y1447.jpg', 'y1105.jpg', 'y1337.jpg', 'y149.jpg', 'y1163.jpg', 'y1190.jpg', 'y169.jpg', 'y198.jpg', 'y1290.jpg', 'y1091.jpg', 'y145.jpg', 'y1120.jpg', 'y1217.jpg', 'y1068.jpg', 'y170.jpg', 'y1008.jpg', 'y1168.jpg', 'y1007.jpg', 'y1169.jpg', 'y1224.jpg', 'y1103.jpg', 'y1016.jpg', 'y1230.jpg', 'y1338.jpg', 'y1460.jpg', 'y1458.jpg', 'y1410.jpg', 'y154.jpg', 'y1152.jpg', 'y1225.jpg', 'y1231.jpg', 'y1075.jpg', 'y1032.jpg', 'y1487.jpg', 'y1052.jpg', 'y1339.jpg', 'y1313.jpg', 'y1426.jpg', 'y1137.jpg', 'y1090.jpg', 'y1089.jpg', 'y1046.jpg', 'y1273.jpg',

In [ ]:
# Build a dataframe        

tumor_df = pd.DataFrame(data=dataset, columns=['Tumor', 'image'])
print(tumor_df.head())
print(tumor_df.tail())

  Tumor                                            image
0   yes  /content/drive/MyDrive/BTdsYesNo//yes/y1333.jpg
1   yes  /content/drive/MyDrive/BTdsYesNo//yes/y1096.jpg
2   yes  /content/drive/MyDrive/BTdsYesNo//yes/y1082.jpg
3   yes   /content/drive/MyDrive/BTdsYesNo//yes/y204.jpg
4   yes  /content/drive/MyDrive/BTdsYesNo//yes/y1408.jpg
     Tumor                                            image
3010    no   /content/drive/MyDrive/BTdsYesNo//no/no246.jpg
3011    no  /content/drive/MyDrive/BTdsYesNo//no/no1194.jpg
3012    no   /content/drive/MyDrive/BTdsYesNo//no/no121.jpg
3013    no   /content/drive/MyDrive/BTdsYesNo//no/no129.jpg
3014    no  /content/drive/MyDrive/BTdsYesNo//no/no1340.jpg


In [ ]:
# Let's check how many samples for each category are present
print("Total number of images are there in the dataset: ", len(tumor_df))

images_count = tumor_df['Tumor'].value_counts()

print("Images in each category: ")
print(images_count)

Total number of images are there in the dataset:  3015
Images in each category: 
no     1509
yes    1506
Name: Tumor, dtype: int64


In [ ]:
import cv2
from PIL import Image



In [ ]:
no_tumor = os.listdir(img_dir + 'no')

#check if it did it perfectly
print(no_tumor)

['no1474.jpg', 'no1058.jpg', 'no291.jpg', 'no1252.jpg', 'no1146.jpg', 'no205.jpg', 'no1375.jpg', 'no1420.jpg', 'no1209.jpg', 'no258.jpg', 'no1172.jpg', 'no1234.jpg', 'no1416.jpg', 'no1359.jpg', 'no1495.jpg', 'no215.jpg', 'no1315.jpg', 'no1158.jpg', 'no1077.jpg', 'no297.jpg', 'no1393.jpg', 'no1186.jpg', 'no127.jpg', 'no125.jpg', 'no1054.jpg', 'no1204.jpg', 'no1262.jpg', 'no1414.jpg', 'no1398.jpg', 'no1282.jpg', 'no1370.jpg', 'no1437.jpg', 'no1415.jpg', 'no1247.jpg', 'no1089.jpg', 'no155.jpg', 'no1391.jpg', 'no1438.jpg', 'no21.jpg', 'no1112.jpg', 'no1109.jpg', 'no1241.jpg', 'no23.jpg', 'no1339.jpg', 'no1144.jpg', 'no158.jpg', 'no1469.jpg', 'no1222.jpg', 'no28.jpg', 'no1400.jpg', 'no181.jpg', 'no1487.jpg', 'no1341.jpg', 'no142.jpg', 'no1108.jpg', 'no1161.jpg', 'no288.jpg', 'no173.jpg', 'no1091.jpg', 'no130.jpg', 'no1351.jpg', 'no1409.jpg', 'no1314.jpg', 'no1394.jpg', 'no222.jpg', 'no1298.jpg', 'no148.jpg', 'no1051.jpg', 'no1463.jpg', 'no1367.jpg', 'no1269.jpg', 'no268.jpg', 'no1082.jpg', 

In [ ]:
im_size = 224

dataset = []
label = []

In [ ]:
for i , img_all in enumerate(no_tumor):
    if( img_all.split('.')[1] == 'jpg' ):
        image = cv2.imread( img_dir + 'no/' + img_all)  # selection of only .jpg format
        image = Image.fromarray( image, 'RGB')
        image = image.resize( (im_size,im_size) ) # resizing image 
        dataset.append( np.array(image))
        label.append(0)

In [ ]:
print(len(label))
len(dataset)

1509


1509

In [ ]:
dataset = np.array(dataset)

dataset = dataset.astype('float32') / 255.0
dataset.shape

(1509, 224, 224, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer


y=tumor_df['Tumor'].values
#print(y[:5])

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
y = y.reshape(-1,1)
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])],remainder='passthrough')
Y=np.array(columnTransformer.fit_transform(y)) #dtype=np.str
Y.shape

(3015, 2)

In [ ]:

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

ValueError: ignored

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
num_classes=2
x = tf.placeholder(tf.float32, shape=[None, 227,227,3])

y_ = tf.placeholder(tf.float32, [None, num_classes])

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:

num_classes=2
weights = {
    'w1': tf.Variable(tf.random_normal([11, 11, 3, 96]),name='w1'),  # 11*11*3   96 filters
    'w2': tf.Variable(tf.random_normal([5, 5, 96, 256]),name='w2'), # 5*5*96  256 filters 
    
    'w3': tf.Variable(tf.random_normal([3, 3, 256, 384]),name='w3'), 
    'w4': tf.Variable(tf.random_normal([3, 3, 384, 384]),name='w4'),
    
    'w5': tf.Variable(tf.random_normal([3, 3, 384, 256]),name='w5'),  
    
    
    'wfc1': tf.Variable(tf.random_normal([6 * 6 * 256, 4096]),name='wfc1'),
    'wfc2': tf.Variable(tf.random_normal([4096, 4096]),name='wfc2'), 
    
    'wout': tf.Variable(tf.random_normal([4096, 2]),name='wout')
}

biases = {
    'b1': tf.Variable(tf.random_normal([96]),name='b1'),
    'b2': tf.Variable(tf.random_normal([256]),name='b2'),
    
    'b3': tf.Variable(tf.random_normal([384]),name='b3'),
    'b4': tf.Variable(tf.random_normal([384]),name='b4'),
    
    'b5': tf.Variable(tf.random_normal([256]),name='b5'),
    
    'bfc1': tf.Variable(tf.random_normal([4096]),name='bfc1'),  #4096
    'bfc2': tf.Variable(tf.random_normal([4096]),name='bfc2'),  #4096
    'bout': tf.Variable(tf.random_normal([num_classes]),name='bout') # 2
}

In [ ]:
def alex_net(x, weights, biases):
  # reshape input to 227*227*3 size
 x = tf.reshape(x, shape=[-1, 227, 227, 3])  
    
 print("###########################################################################")
 print("size of x is")
 print(x.shape)
    


 #conv1
 #kernel size=11*11, stride=4
 
 conv1_in = tf.nn.conv2d(x, weights['w1'], strides=[1, 4, 4, 1], padding="SAME")
 conv1_in = tf.nn.bias_add(conv1_in,  biases['b1'])   # y= xw+b
 conv1 = tf.nn.relu(conv1_in)
 #maxpool1   #kernel size 3*3, stride 2

 maxpool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
 print("###########################################################################")
 print("size after 1st conv layer) is ")
 print(maxpool1.shape)
    

 #conv2
 # kernel 5*5, channels 256, stride 1
 conv2_in = tf.nn.conv2d(maxpool1,weights['w2'], strides=[1, 1, 1, 1], padding="SAME")
 conv2_in = tf.nn.bias_add(conv2_in,  biases['b2'])   
 conv2 = tf.nn.relu(conv2_in)

 #maxpool2  
 #kernel 3*3 stride 2
 maxpool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
 print("###########################################################################")
 print("size after 2nd conv layer) is ")
 print(maxpool2.shape)
  
 #conv3
 #kernel size=3*3, channels=384, stride 1   
 conv3_in = tf.nn.conv2d(maxpool2, weights['w3'],strides=[1, 1, 1, 1], padding="SAME")
 conv3_in = tf.nn.bias_add(conv3_in,  biases['b3'])  
 conv3 = tf.nn.relu(conv3_in)
 print("###########################################################################")
 print("size after 3rd conv layer) is ")
 print(conv3.shape)
     

 #conv4
 # kernel=3*3,channels=384,stride=1   
 conv4_in = tf.nn.conv2d(conv3, weights['w4'], strides=[1, 1, 1, 1], padding="SAME")
 conv4_in = tf.nn.bias_add(conv4_in,  biases['b4'])  
 conv4 = tf.nn.relu(conv4_in)
 print("###########################################################################")
 print("size after 4th conv layer) is ")
 print(conv4.shape)

 #conv5
 # kernel 3*3, channels=256, stride 1   
 conv5_in = tf.nn.conv2d(conv4, weights['w5'],strides=[1, 1, 1, 1], padding="SAME")
 conv5_in = tf.nn.bias_add(conv5_in,  biases['b5']) 
 conv5 = tf.nn.relu(conv5_in)
 
 #maxpool5  # kernel 3*3, stride 2
 maxpool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

 print("###########################################################################")
 print("size after 5th conv layer) is ")
 print(conv5.shape)
 print("###########################################################################")
 print("size after 5th conv layer pooling) is ")
 print(maxpool5.shape)


 fc6 = tf.reshape(maxpool5, [-1, weights['wfc1'].get_shape().as_list()[0]])
 print("###########################################################################")
 print("size after reshaping the image ) is ")
 print(fc6.shape) 

 fc6 = tf.add(tf.matmul(fc6, weights['wfc1']), biases['bfc1'])
 fc6 = tf.nn.relu(fc6)
      

 #fc7
 fc7 = tf.nn.relu_layer(fc6, weights['wfc2'], biases['bfc2'])
 print("###########################################################################")
 print("size after 7th layer) is ")
 print(fc7.shape)


 fc8 = tf.add(tf.matmul(fc7, weights['wout']), biases['bout'])
 out = tf.nn.softmax(fc8)
    
 return out   

 

In [ ]:
# Create the model
model = alex_net(x, weights, biases)
print(model)

###########################################################################
size of x is
(?, 227, 227, 3)
###########################################################################
size after 1st conv layer) is 
(?, 28, 28, 96)
###########################################################################
size after 2nd conv layer) is 
(?, 13, 13, 256)
###########################################################################
size after 3rd conv layer) is 
(?, 13, 13, 384)
###########################################################################
size after 4th conv layer) is 
(?, 13, 13, 384)
###########################################################################
size after 5th conv layer) is 
(?, 13, 13, 256)
###########################################################################
size after 5th conv layer pooling) is 
(?, 6, 6, 256)
###########################################################################
size after reshaping the image ) is 
(?, 9216)
######################

In [ ]:
learning_rate=0.01
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
init = tf.initialize_all_variables()


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
cost_history=[]
n_epochs =5
# the execution
sess = tf.Session()
sess.run(init)

train_y=train_y.todense()


for i in range(n_epochs):
    a, c = sess.run([optimizer, cost], feed_dict={x: train_x, y_: train_y})  #working
    cost_history = np.append(cost_history,c)  # working
    print('epoch : ', i,  ' - ', 'cost: ', c) #working 


NameError: ignored